# Imports

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns

from sklearn import datasets, metrics, model_selection
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, RandomizedSearchCV, KFold

import lightgbm as lgb

import tensorflow
from tensorflow import keras
# from plot_keras_history import plot_history

import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest

C:\Users\celia\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\celia\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\celia\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\celia\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

# Pré-traitement train 

In [2]:
# Load datasets
df_train = pd.read_csv("X_station_train.csv",parse_dates=['date'],infer_datetime_format=True)
coords   = pd.read_csv("stations_coordinates.csv")
Y_train  = pd.read_csv("Y_train.csv",parse_dates=['date'],infer_datetime_format=True)

In [3]:
# On clip les outliers
def find_outliers(series):
    return (series - series.mean()) > 1.5 * series.std() # ou 2.4


def cap_values(series):
    outliers = find_outliers(series)
    max_val = series[~outliers].max()
    print(max_val)
    series[outliers] = max_val
    return series

In [4]:
def treat_train(coords_df, train, train_Y):
    # Merge coords + train 
    coords_df['number_sta'] = coords_df['number_sta'].astype('category')
    df_X_train = train.merge(coords, on=['number_sta'], how='left')
    
    # Traitement date
    df_X_train["date_wh"] = df_X_train["date"].apply(lambda x: dt.date(x.year, x.month, x.day))
    
    # Traitement NaN
    # Premier remplissage par station et date
    df_X_train = df_X_train.groupby(['number_sta','date'], sort=False).apply(lambda x: x.ffill().bfill())

    # Remplissage des données manquantes restantes par l'algorithme MissForest
    #imputation_train = MissForest()
    #df_X_train = imputation_train.fit_transform(df_X_train)
    
    # Moyenne des variables groupées par la date et la station
    sub_df1 = df_X_train[['date_wh', 'number_sta',"ff", "t", "td", "hu", "dd"]].groupby(['date_wh', 'number_sta']).mean().reset_index()

    # Somme des précipitations sur la date et la station
    sub_df2 = df_X_train[['date_wh', 'number_sta',"precip"]].groupby(['date_wh', 'number_sta']).sum().reset_index()

    # Récupération des latitudes, logitudes, hauteurs des stations 
    sub_df3 = df_X_train[['date_wh', 'number_sta',"lat", "lon", "height_sta"]].drop_duplicates(['date_wh', 'number_sta'])

    # Fusion des 3 sub_sets
    df_X_train = sub_df1.merge(sub_df2, on = ["date_wh", "number_sta" ], how = "left")
    df_X_train = df_X_train.merge(sub_df3, on = ["date_wh", "number_sta" ], how = "left")
    df_X_train = df_X_train.rename(columns = {"date_wh": "date"})
    df_X_train.reset_index()
    
    # Ajout colonne Y_train
    train_Y['number_sta'] = train_Y['number_sta'].astype('category')
    df_X_train['number_sta'] = df_X_train['number_sta'].astype('category')
    train_Y["date"] = train_Y["date"].apply(lambda x: x- dt.timedelta(days=1))
    df_X_train['date'] = df_X_train['date'].apply(lambda x: pd.to_datetime(x, format='%Y-%m-%d'))
    df_X_train = pd.merge(df_X_train, train_Y, on = ["number_sta", "date"], how = "left")
    
    # Ajout du mois 
    df_X_train["month"] = df_X_train["date"].apply(lambda x: x.month)
    # Transformation en variable facteur
    df_X_train['month'] = pd.Categorical(df_X_train['month'], ordered=False)
    
    # Gestion des données manquantes de la valeur à prédire
    df_X_train.dropna(inplace = True) # on supprime les lignes dont la valeur exacte à prédire est manquante
    df_X_train.drop(["date", "number_sta", "Id"], axis = 1, inplace = True)
    
    # Variable month 
    df_X_trainDum = pd.get_dummies(df_X_train[["month"]])
    del df_X_trainDum["month_1"] 
    df_X_train.drop('month', axis=1, inplace=True)
    df_X_train =pd.concat([df_X_train, df_X_trainDum,],axis=1)
    
    # Clip values
    capped_values = cap_values(df_X_train["Ground_truth"])
    df_X_train.drop(["Ground_truth"], axis = 1, inplace=True)
    df_X_train["Ground_truth"] = capped_values
    
    return df_X_train

In [ ]:
df_X_train = treat_train(coords, df_train, Y_train)

In [ ]:
df_X_train.to_csv("df_train_nan_by_station_date.csv", index = False)

# Pré-traitement test

In [8]:
# Load datasets
df_test = pd.read_csv("X_station_test.csv")
#test_forecast = pd.read_csv("Baseline_forecast_test.csv")
test_obs = pd.read_csv("Baseline_observation_test.csv")

In [ ]:
def pre_test(test_df, obs_test, coords_df):
    # Number_sta + merge coords
    df_X_test = test_df.copy() 
    coords_df['number_sta'] = coords_df['number_sta'].astype('category')
    df_X_test["number_sta"] = df_X_test["Id"].apply(lambda x : x.split("_")[0])
    df_X_test["number_sta"] = df_X_test["number_sta"].astype("int")
    df_X_test = df_X_test.merge(coords_df, on=["number_sta"], how = "left")
    
    # Création variable day 
    df_day = df_X_test["Id"].apply(lambda x : x.split("_")[1])
    df_X_test.insert(0,"day", df_day)
    df_X_test["day"] = df_X_test["day"].astype("int")
    # test -> changement en facteur
    df_X_test['day'] = pd.Categorical(df_X_test['day'], ordered=False)
    
    # Sort by number_sta and day 
    df_X_test.sort_values(by = ['number_sta', 'day'], inplace = True)
    
    # Fill na
    df_X_test = df_X_test.groupby(['number_sta','day'], sort=False).apply(lambda x: x.ffill().bfill())
    #imputation = MissForest()
    #df_X_test = imputation.fit_transform(df_X_test)
    
    # Moyenne des variables groupées par la date et la station
    sub_df1 = df_X_test[['day', 'number_sta',"ff", "t", "td", "hu", "dd"]].groupby(['day', 'number_sta']).mean().reset_index()

    # Somme des précipitations sur la date et la station
    sub_df2 = df_X_test[['day', 'number_sta',"precip"]].groupby(['day', 'number_sta']).sum().reset_index()

    # Récupération des latitudes, logitudes, hauteurs des stations, du mois et de l'Id
    sub_df3 = df_X_test[['day', 'number_sta',"lat", "lon", "height_sta", "month", "Id"]].drop_duplicates(['day', 'number_sta'])

    # Fusion des 3 sub_sets
    df_X_test = sub_df1.merge(sub_df2, on = ["day", "number_sta" ], how = "left")
    df_X_test = df_X_test.merge(sub_df3, on = ["day", "number_sta" ], how = "left")
    df_X_test = df_X_test.rename(columns = {"day": "date"})
    df_X_test.reset_index()

    # Sauvegarde des Ids sous le bon format pour plus tard
    id_list = pd.DataFrame(data = df_X_test["Id"].apply(lambda x : x.split("_")[0] + '_' + x.split("_")[1]))
    df_X_test.drop(['Id'], axis=1, inplace=True)
    df_X_test['Id'] = id_list

    # Changement du type de la variable month en facteur 
    df_X_test['month'] = pd.Categorical(df_X_test['month'], ordered=False)

    df_X_testDum = pd.get_dummies(df_X_test[["month"]])

    del df_X_testDum["month_1"] 

    # Variables explicatives quantitatives
    df_X_testQuant= df_X_test.drop('month', axis=1, inplace=False)

    # Variables explicatives
    df_X_test = pd.concat([df_X_testQuant, df_X_testDum,],axis=1)
    
    # On ne conserve que les Id qui sont dans la liste de la baseline (Id souhaités pour la prédiction)
    df_X_test = df_X_test[df_X_test.Id.isin(test_obs['Id'])]
    
    return df_X_test

In [ ]:
df_X_test = pre_test(df_test, test_obs, coords)

In [ ]:
df_X_test.to_csv("df_test_nan_by_station_day.csv", index = False)